<a href="https://colab.research.google.com/github/SamuelWelz/MachineLearning/blob/main/ML_Assignment_RAG_System%2BAgentensystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installation der benötigten Pakete

In [2]:
# Installation der benötigten Python-Pakete in Google Colab.
# Das Ausrufezeichen (!) führt Terminalbefehle in Colab aus, hier z. B. pip zur Installation.

# LangChain-Module für die Verwendung von OpenAI-LLMs und Community-Tools:
# - `langchain-openai`: Stellt einfache Python-Klassen bereit, um OpenAI-Modelle (z. B. GPT-4o)
#    anzusprechen. Die komplexe HTTP-Kommunikation mit der OpenAI-API (Authentifizierung, Request-Format,
#    Response-Parsing) wird vollständig abstrahiert – wir müssen nur noch .invoke() aufrufen.
# - `langchain-community`: Enthält viele Dokumenten-Loader (z. B. für PDFs, Webseiten, CSVs)
#    und Integrationen mit Vektordatenbanken wie FAISS, Pinecone etc.
!pip install langchain-openai langchain-community

# `pypdf`: Eine Hilfsbibliothek zur Verarbeitung von PDF-Dateien.
# Wird von LangChain intern verwendet, um PDFs seitenweise zu lesen.
# Ohne diese Bibliothek wäre ein manuelles Parsen des PDF-Inhalts nötig.
!pip install pypdf

# `faiss-cpu`: Installiert die CPU-Version der Vektor-Datenbank FAISS von Meta AI.
# Damit kann man lokal Vektoren (z. B. Embeddings) effizient speichern und durchsuchen.
# LangChain abstrahiert die native C++-FAISS-Schnittstelle vollständig,
# sodass man mit Python-Objekten wie `FAISS.from_documents(...)` arbeiten kann.
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.1 MB/s eta 0:00:00


# 2. Alle wichtigen Importe

In [3]:
# Zugriff auf Secrets in Google Colab (API-Key)
# ----------------------------------------------
# `userdata.get("apikey_openai")` greift sicher auf einen in Colab hinterlegten geheimen API-Key zu.
# Vorteil: Der API-Key ist nicht im Code sichtbar (Datenschutz!), sondern wurde vorher manuell unter dem Schlüssel-Icon links in Colab eingetragen.
from google.colab import userdata
OPENAI_API_KEY = userdata.get("apikey_openai")

# Zugriff auf OpenAI-Modelle (LLM)
# --------------------------------
# `ChatOpenAI` ist eine LangChain-Klasse, die das Senden von Prompts an OpenAI-Modelle stark vereinfacht.
# → Wegabstrahiert werden:
#   - die direkte HTTP-Kommunikation mit der OpenAI-API
#   - das Serialisieren des Prompts
#   - das Parsen der JSON-Antwort
#   - Fehlerbehandlung (Rate Limits etc.)
#   - das Token-Management
# Der Entwickler arbeitet nur noch mit einer einfachen Python-Klasse und ruft `.invoke()` auf.

from langchain_openai.chat_models import ChatOpenAI

# Zugriff auf OpenAI-Embeddings
# -----------------------------
# `OpenAIEmbeddings` erzeugt Vektorrepräsentationen (sog. Embeddings) für Texte.
# → Wegabstrahiert:
#   - API-Kommunikation mit OpenAI (Endpoint für Embeddings)
#   - Formatierung der Eingabetexte
#   - Rückgabe der Embedding-Vektoren (Listen aus Floats)
#   - Wiederholung bei API-Fehlern oder zu langen Texten
# Diese Embeddings sind zentral für das spätere Retrieval (Vektorsuche).
from langchain_openai.embeddings import OpenAIEmbeddings

# PDF-Dateien laden und in lesbaren Text umwandeln
# ------------------------------------------------
# `PyPDFLoader` ist ein Document Loader aus LangChain Community.
# Er kapselt:
#   - das Parsen der PDF-Datei (basierend auf `pypdf`)
#   - das Extrahieren der Texte aus jeder Seite
#   - das Umwandeln der Seiten in `Document`-Objekte von LangChain
# Ohne diese Abstraktion müsste man sich mit `pypdf`, Textextraktion, Fehlern etc. selbst beschäftigen.
from langchain_community.document_loaders import PyPDFLoader

# Text in sinnvolle Abschnitte (Chunks) teilen
# --------------------------------------------
# `RecursiveCharacterTextSplitter` ist ein Utility, das Text in semantisch sinnvolle Stücke aufteilt,
# wobei z. B. auf Satz- oder Absatzgrenzen geachtet wird.
# → Wegabstrahiert:
#   - eigene Logik zum Splitten langer Texte in passende Häppchen
#   - Verwaltung von Overlap zwischen Chunks (z. B. 100 Token Overlap)
#   - Handling von Grenzen bei Wörtern, Absätzen etc.
# Dies ist entscheidend für sinnvolle Embedding-Vektoren und präzise semantische Suche.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# FAISS als Vektor-Datenbank
# --------------------------
# `FAISS` ist ein schneller In-Memory-Vectorstore für Ähnlichkeitssuchen.
# Die LangChain-Integration `langchain_community.vectorstores.FAISS` kapselt:
#   - die Generierung von Vektoren für Dokument-Chunks
#   - das Indexieren der Vektoren
#   - die Suche nach den ähnlichsten Chunks für eine Query
#   - die Rückgabe relevanter Dokumente anhand der Embedding-Ähnlichkeit
# Dadurch entfällt das manuelle Arbeiten mit Vektor-Matrizen und KNN-Algorithmen.
from langchain_community.vectorstores import FAISS

# Prompt-Vorlagen (Templates) erstellen
# -------------------------------------
# `ChatPromptTemplate` erlaubt es, strukturierte Prompts mit Platzhaltern zu erstellen.
# Die Abstraktion übernimmt:
#   - das Einsetzen von Variablen wie {context}, {question}
#   - das sichere Formatieren ohne Zeichenprobleme
#   - das Vorbereiten des finalen Prompts zur Übergabe ans LLM
from langchain.prompts import ChatPromptTemplate

# LangChain Runnable: Pipe-Element, das einfach Daten durchreicht
# ----------------------------------------------------------------
# `RunnablePassthrough()` wird später in der Chain genutzt, um eine Frage unverändert durchzuleiten.
# Es ist ein Baustein in der modularen LangChain-Architektur (jede Komponente ist eine „runnable pipeline unit“).
from langchain_core.runnables import RunnablePassthrough

# Output-Parser
# -------------
# `StrOutputParser` wandelt das Roh-Ergebnis eines LLM-Aufrufs (vom Typ AIMessage) in einen reinen String um.
# Abstraktion:
#   - entfernt technische Metadaten aus der Antwort
#   - sorgt dafür, dass am Ende ein sauberer Antwort-Text steht, mit dem man weiterarbeiten kann
from langchain_core.output_parsers import StrOutputParser

# 3. PDF laden

In [6]:
# PDF-Datei aus GitHub laden
# ----------------------------------------------------------
# Der bisherige Ansatz mit `pdf_path` funktioniert nur für lokale Dateien im Colab-Dateisystem.
# Bei einem GitHub-Link (z. B. https://github.com/...) wird jedoch nicht direkt die PDF,
# sondern eine HTML-Webseite geladen, die die PDF nur anzeigt.
# Für die Verarbeitung wird der direkte RAW-Content der Datei benötigt.

import requests
import tempfile
from langchain_community.document_loaders import PyPDFLoader

# URL zum PDF im GitHub-Repository (RAW-Dateiversion verwenden!)
# Der Permalink aus GitHub muss in den "raw.githubusercontent.com"-Link umgewandelt werden.
pdf_url = "https://raw.githubusercontent.com/SamuelWelz/MachineLearning/a68810309c535ece8c4e8bb6ed95d392321323c7/BitcoinMining.pdf"

# PDF von GitHub herunterladen
response = requests.get(pdf_url)
response.raise_for_status()  # Falls Download fehlschlägt, wird ein HTTPError ausgelöst.

# Temporäre Datei anlegen und PDF-Inhalt dort speichern
# Hintergrund: PyPDFLoader benötigt einen Dateipfad, nicht direkt die Binärdaten.
with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
    tmp_file.write(response.content)
    temp_pdf_path = tmp_file.name  # Pfad zur temporären PDF-Datei

# PDF laden mit dem LangChain PDF-Loader
# --------------------------------------
# PyPDFLoader ist ein DocumentLoader aus der LangChain Community-Bibliothek.
# Er abstrahiert intern:
#   1. Öffnen der PDF-Datei über pypdf
#   2. Extraktion des Textinhalts aus jeder einzelnen Seite
#   3. Umwandlung in eine Liste von Document-Objekten (ein Objekt pro Seite),
#      wobei jedes Objekt `page_content` (Text) und `metadata` (z. B. Seitenzahl) enthält.
loader = PyPDFLoader(temp_pdf_path)

# `.load()` führt den eigentlichen Lese- und Extraktionsprozess aus.
# Ergebnis: Liste von Document-Objekten, die als Eingabe für Chunking/Embedding genutzt werden können.
documents = loader.load()

# 4. Dokument splitten in Chunks

In [7]:
# Dokument in gleichgroße Text-Chunks aufteilen
# ---------------------------------------------
# LangChain erwartet für Embedding- und Retrieval-Anwendungen kleinere,
# sinnvoll segmentierte Abschnitte, sogenannte "Chunks".
# Große Dokumente (wie lange PDF-Seiten) müssen daher in kleinere Stücke
# unterteilt werden, damit:
# - das Token-Limit von LLMs nicht überschritten wird
# - relevante Informationen besser extrahiert werden können

# Der RecursiveCharacterTextSplitter übernimmt das automatische "Chunking".
# Dabei passiert im Hintergrund folgendes:
# 1. Das Dokument wird rekursiv entlang vordefinierter Trennzeichen (Absatz, Satz, Wort) geteilt.
#    Falls der Text zu groß ist, wird auf feinere Trennmethoden zurückgegriffen.
# 2. Der `chunk_size` legt die Zielgröße der Chunks in Zeichen fest (hier: 250).
# 3. Der `chunk_overlap` sorgt dafür, dass sich die Chunks überlappen (hier: 75 Zeichen).
#    → Dadurch bleiben zusammenhängende Gedanken auch bei Chunk-Grenzen erhalten
#      und Kontext wird robuster vermittelt (wichtig für Embedding & Retrieval).

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,       # Jeder Chunk enthält max. 250 Zeichen
    chunk_overlap=75      # Davon überlappen sich 75 Zeichen mit dem nächsten Chunk
)

# Die Methode `.split_documents()` verarbeitet die LangChain-Document-Objekte
# und erzeugt daraus eine Liste von kleineren Dokumenten-Chunks
# → Jeder Eintrag in `split_documents` ist ein eigenständiger `Document` mit eigenem Textabschnitt

split_documents = text_splitter.split_documents(documents)

# 5. Embeddings erzeugen und in Vectorstore speichern

In [8]:
# Embedding-Modell definieren
# ----------------------------
# Definiert ein Embedding-Modell von OpenAI, das dazu verwendet wird,
# um Text-Chunks in semantische Vektor-Repräsentationen zu konvertieren.
#
# Eingesetztes Modell: "text-embedding-3-small"
# - Liefert numerische Vektoren, die semantische Ähnlichkeit zwischen Texten abbilden
# - Eignet sich für effiziente Ähnlichkeitssuche im Retrieval-Schritt
#
# Abstrahierung durch LangChain:
# - Automatischer API-Aufruf zur OpenAI-Schnittstelle
# - Automatische Tokenisierung des Textes und Rückgabe der Embeddings als Listen von Float-Werten
# - Kein manuelles Handling der HTTP-Anfragen oder JSON-Parsing erforderlich

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)


# Vectorstore erstellen (FAISS lokal im Arbeitsspeicher)
# -------------------------------------------------------
# Erstellt einen Vektorindex aus den zuvor erzeugten Text-Chunks und den zugehörigen Embeddings.
# Verwendet wird FAISS (Facebook AI Similarity Search), ein Framework für effiziente
# Ähnlichkeitssuche in hochdimensionalen Vektorräumen.
#
# Hintergrundprozesse:
# - Jeder Text-Chunk wird vom Embedding-Modell in einen Vektor umgewandelt
# - Die resultierenden Vektoren werden in FAISS gespeichert und indiziert
# - Der Index ermöglicht spätere semantische Suchen über `similarity_search()`
#
# Vorteile durch LangChain:
# - Kein manueller Aufbau des FAISS-Index notwendig
# - Speicherung der Dokumente und Vektoren erfolgt transparent
# - Zugriff auf Inhalte durch Retrieval-Methoden mit automatischer Relevanzbewertung

vectorstore = FAISS.from_documents(split_documents, embedding=embeddings)


# 6. Prompt-Vorlage definieren

In [9]:
from langchain.prompts import ChatPromptTemplate

# Prompt-Template definieren
# ---------------------------
# Erstellt eine strukturierte Prompt-Vorlage für das LLM, die aus zwei Komponenten besteht:
# 1. Kontext (aus der Vektordatenbank)
# 2. Frage (vom Nutzer gestellt)
#
# Der Prompt stellt sicher, dass das Modell die Frage auf Basis des Kontextes beantwortet
# und im Zweifelsfall eine standardisierte Aussage trifft ("Ich weiß es nicht").

template = """
Beantworte die Frage basierend auf dem gegebenen Kontext.
Wenn du keine Antwort findest, sag bitte: "Ich weiß es nicht."

Context: {context}

Question: {question}
"""

# Prompt-Objekt erstellen
# ------------------------
# Die Methode `from_template()` abstrahiert mehrere manuelle Schritte:
# - Ersetzt Platzhalter {context} und {question} automatisch zur Laufzeit
# - Verpackt den Prompt als standardisiertes LangChain-Objekt
# - Unterstützt spätere Komposition in Chains (z. B. | Prompt | Modell)
#
# Vorteil:
# - Kein manuelles Prompt-Building mit String-Konkatenation notwendig
# - Bessere Wiederverwendbarkeit und Klarheit der Prompt-Struktur

prompt = ChatPromptTemplate.from_template(template)

# 7. OpenAI LLM & Parser vorbereiten

In [44]:
# Die Klasse `ChatOpenAI` kapselt den Zugriff auf die OpenAI-API und übernimmt mehrere Schritte:
# - Aufbau und Verwaltung der HTTP-Verbindung zur OpenAI API
# - Formatierung der Anfrage (z. B. Modellwahl, Parameter wie Temperatur und Tokenlimit)
# - Verwaltung der Modellantwort (inkl. Token-Handling, Logging, Fehlerbehandlung)
#
# Vorteil:
# - Keine manuelle API-Nutzung mit HTTP-Requests oder JSON erforderlich
# - Einheitliche Schnittstelle für alle unterstützten OpenAI-Modelle
# - Direkte Integration in LangChain-Pipelines möglich

# GPT-4o-mini verwenden
model = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",
    temperature=0.2,    # Steuert die Kreativität des Modells. Niedriger Wert führt zu deterministischeren, prägnanteren Antworten.
    max_tokens=200      # Begrenzung der maximalen Antwortlänge, um den Tokenverbrauch zu kontrollieren und Kosten zu senken.
)


# Parser: Nur der reine Antworttext soll extrahiert werden
parser = StrOutputParser()

# Der `StrOutputParser` ist ein Output-Parser, der die Antwort des Modells bereinigt:
# - Extrahiert nur den tatsächlichen Antwortinhalt aus dem vollständigen `AIMessage`-Objekt
# - Entfernt Metadaten, Tokeninformationen oder andere technische Elemente
#
# Vorteil:
# - Die Ausgabe kann direkt weiterverwendet oder angezeigt werden
# - Kein manuelles Parsen der Rückgabe erforderlich

# 8. Chain bauen

In [11]:
# Aufbau der vollständigen Retrieval-Augmented Generation (RAG)-Pipeline
# -------------------------------------------
# Diese Chain kombiniert alle zuvor definierten Schritte zu einem automatisierten Ablauf:
# 1. Die Nutzerfrage wird an den Retriever übergeben.
# 2. Der Retriever durchsucht den Vektorstore nach passenden Textstellen (Chunks) und gibt diese als Kontext zurück.
# 3. Kontext und Frage werden in einem Prompt-Template kombiniert.
# 4. Der formattierte Prompt wird an das Sprachmodell (LLM) übergeben.
# 5. Die Antwort des LLM wird durch einen Parser in reinen Text umgewandelt.
# LangChain abstrahiert dabei viele technische Details wie Embedding-Suche, Prompt-Zusammenbau oder Modellkommunikation.

from langchain_core.runnables import RunnablePassthrough

# Retriever-Chain aufbauen
chain = (
    {
        "context": vectorstore.as_retriever(),       # Der Vektorstore wird in einen Retriever umgewandelt, der bei einer Anfrage relevante Chunks zurückliefert
        "question": RunnablePassthrough()            # Übergibt die Frage unverändert weiter, da sie nicht verändert oder vorverarbeitet werden muss
    }
    | prompt                                         # Kombiniert Kontext und Frage gemäß der Prompt-Vorlage zu einem vollständigen Prompt
    | model                                          # Sendet den formatierten Prompt an das Sprachmodell (z. B. GPT-4o-mini)
    | parser                                         # Filtert aus der Modellantwort den reinen Antworttext heraus
)


# 9. Testlauf der Pipeline

In [45]:
# Definition einer Beispiel-Frage, die an das RAG-System gestellt wird.
frage = "Was haben Gewächshäuser mit BTC-Minig zu tun?"

# Weitere mögliche Fragen (zur Demonstration/Tests):
# - Was gibt es für Arten von BTC-Mining?
# - Was ist Bitcoin-Mining?

# Die gesamte Chain wird durch .invoke(frage) ausgeführt:
# Intern passiert dabei Folgendes:
# 1. Die Frage wird an den Retriever übergeben.
# 2. Der Retriever berechnet ein Embedding der Frage.
# 3. Dieses Embedding wird mit den Vektoren im Vektorstore verglichen
# 4. Die ähnlichsten Chunks (Kontext) werden selektiert und gemeinsam mit der Frage an das Prompt-Template übergeben.
# 5. Das resultierende Prompt wird an das OpenAI-Modell gesendet.
# 6. Das Modell erzeugt eine Antwort.
# 7. Der Output-Parser entfernt technische Metadaten und gibt den reinen Antworttext zurück.

antwort = chain.invoke(frage)

# Ausgabe der modellgenerierten Antwort zur Kontrolle
print("Antwort:\n", antwort)

Antwort:
 Gewächshäuser können mit BTC-Mining in Verbindung stehen, indem die Wärme, die von Mining-Servern erzeugt wird, genutzt wird, um das Gewächshaus zu heizen. Dadurch entstehen "echte Bitcoin-Blumen", und die Energie für die Mining-Hardware kann aus Solarzellen stammen, die auf dem Dach des Gewächshauses installiert sind. Dies schafft einen umweltfreundlichen Mining-Prozess, der die Energieerzeugung und -weitergabe effizient nutzt.


# --------------------- AGENTENSYSTEM ---------------------

# Imports & Guards

In [61]:
# Importiert Standardtypen für Typannotationen von Listen
from typing import List

# Importiert Pydantic-Basisklasse und Field, um Datenmodelle mit Validierung zu erstellen
from pydantic import BaseModel, Field

# Versucht zuerst den aktuellen LangChain-OpenAI-Import, fällt bei älteren Versionen auf den alten Pfad zurück
try:
    from langchain_openai import ChatOpenAI
except Exception:
    from langchain_openai.chat_models import ChatOpenAI  # Fallback für ältere Versionen

# Importiert Hilfsklasse zum Erstellen strukturierter Chat-Prompt-Vorlagen mit Platzhaltern
from langchain.prompts import ChatPromptTemplate

# Importiert den Tool-Dekorator, um Python-Funktionen als Werkzeuge für Agents nutzbar zu machen
from langchain.tools import tool

# Importiert Klassen, um Agents mit Tools zu bauen und auszuführen
from langchain.agents import AgentExecutor, create_openai_tools_agent

# Stellt sicher, dass der API-Key und der Vektorindex (vectorstore) bereits aus dem RAG-Setup existieren
assert 'OPENAI_API_KEY' in globals() and OPENAI_API_KEY, "OPENAI_API_KEY fehlt. Bitte RAG-Setup-Zellen ausführen."
assert 'vectorstore' in globals(), "vectorstore fehlt. Bitte RAG-Setup-Zellen ausführen (FAISS-Index)."

# Erstellt das Basismodell für die OpenAI-Chat-API mit geringer Temperatur für deterministische Antworten
# max_tokens wird hier bewusst nicht gesetzt, um eine spätere flexible Anpassung zu erlauben
base_model = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",
    temperature=0.2,
)

# Bindet ein hartes Completion-Limit an das Modell, damit alle Aufrufe max. 500 Tokens zurückgeben
model = base_model.bind(max_completion_tokens=500)

# Konsolenausgabe, um zu bestätigen, dass Modell und Vectorstore bereitstehen
print("OK – model und vectorstore verfügbar.")


OK – model und vectorstore verfügbar.


# Tools (RAG + Calculator)

In [64]:
# ===== 1) TOOLS =====
# In diesem Abschnitt werden zwei eigene Tools definiert, die später vom Agenten
# automatisch aufgerufen werden können.
# Tools sind in LangChain gekapselte Python-Funktionen, die mit zusätzlichen
# Metadaten (Name, Beschreibung) versehen werden.
# Sie ermöglichen es einem Agenten, während der Laufzeit gezielt externe Funktionen
# zu nutzen
#
# Tool 1: "rag_search"
# --------------------
# Zweck:
# - Führt eine semantische Suche im FAISS-Vectorstore durch
# - Gibt die Top-k relevantesten Text-Snippets aus dem Dokument zurück
#
# Ablauf:
# 1. Nimmt eine Suchanfrage (query) entgegen
# 2. Führt mit `vectorstore.similarity_search()` eine Vektorsuche aus
# 3. Kürzt die Ergebnisse mit einer internen Hilfsfunktion `_short()`, um die Tokenanzahl
#    zu reduzieren (max. 350 Zeichen pro Treffer)
# 4. Baut eine strukturierte Ergebnisliste auf (Nummer, Seitenzahl, Quelle, Text)
# 5. Gibt alles als String zurück
#
# Vorteil:
# - Der Agent muss nicht wissen, wie FAISS funktioniert oder wie die Suche implementiert ist.
# - Die gesamte Logik (Suche, Kürzen, Formatieren) ist gekapselt und wiederverwendbar.

@tool("rag_search")
def rag_search(query: str, k: int = 2) -> str:  # war k=4
    """Durchsucht den FAISS-Index und liefert kompakte Top-k Snippets."""
    docs = vectorstore.similarity_search(query, k=k)

    def _short(text, n=350):  # Kürzt lange Snippets
        t = (text or "").strip().replace("\n", " ")
        return (t[:n] + "…") if len(t) > n else t

    lines = []
    for i, d in enumerate(docs, start=1):
        meta = d.metadata if isinstance(d.metadata, dict) else {}
        src = meta.get("source", "PDF")
        page = meta.get("page", "?")
        lines.append(f"[{i}] (S.{page}, {src}) {_short(d.page_content)}")

    return "\n".join(lines)


# Tool 2: "calc"
# --------------
# Zweck:
# - Führt sichere mathematische Berechnungen mit Python durch, ohne `eval` zu verwenden
#
# Ablauf:
# 1. Parst den Ausdruck mit `ast.parse()` in einen abstrakten Syntaxbaum (AST)
# 2. Erlaubt nur definierte Operatoren (+, -, *, /, Potenz, Vorzeichen)
# 3. Bewertet den Ausdruck rekursiv mit `_eval()`
# 4. Gibt das Ergebnis als String zurück oder im Fehlerfall eine Meldung
#
# Vorteil:
# - Sicherheit: Kein direkter Code-Execution-Risiko wie bei `eval`
# - Komplette Kapselung der Parsing- und Berechnungslogik in einer einfachen Funktion
#
# Am Ende werden beide Tools in einer Liste `tools` gesammelt,
# sodass sie später beim Erstellen des Agenten registriert werden können.

@tool("calc")
def calc(expression: str) -> str:
    """Einfacher Rechner für + - * / ** und Klammern. Beispiel: '2*(3+4)/5'."""
    import ast, operator as op

    allowed_ops = {
        ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
        ast.Div: op.truediv, ast.Pow: op.pow, ast.USub: op.neg
    }

    def _eval(node):
        if isinstance(node, ast.Num): return node.n
        if isinstance(node, ast.BinOp): return allowed_ops[type(node.op)](_eval(node.left), _eval(node.right))
        if isinstance(node, ast.UnaryOp): return allowed_ops[type(node.op)](_eval(node.operand))
        if isinstance(node, ast.Expression): return _eval(node.body)
        raise ValueError("Nicht erlaubter Ausdruck")

    try:
        node = ast.parse(expression, mode="eval")
        return str(_eval(node))
    except Exception as e:
        return f"Fehler: {e}"


tools = [rag_search, calc]
print("OK – Tools registriert:", [t.name for t in tools])

OK – Tools registriert: ['rag_search', 'calc']


# Planner (Structured Output, Schritt 1 im Chaining)

In [66]:
# ===== 2) PLANNER =====
# Der Planner ist die erste Stufe in unserer Agenten-Pipeline.
# Er hat die Aufgabe, eine unstrukturierte Nutzerfrage in einen klaren, maschinenlesbaren Plan zu übersetzen.
# Dieser Plan wird als JSON im Format unseres Pydantic-Modells `Plan` zurückgegeben.
#
# 1. Pydantic-Datenmodell `Plan`
# ------------------------------
# Das Modell beschreibt, welche Felder der Planner füllen soll:
# - intent: Kurzbeschreibung der Aufgabe (z. B. "Erklärung schreiben", "Berechnung durchführen")
# - needs_retrieval: Boolean, ob für die Antwort Kontext aus der RAG-Suche benötigt wird
# - needs_calculation: Boolean, ob eine mathematische Berechnung nötig ist
# - queries: Liste mit Suchbegriffen für die RAG-Suche
# - calc_expressions: Liste mit Mathe-Ausdrücken, die ggf. im calc-Tool berechnet werden sollen
# - subtasks: Liste mit einzelnen Arbeitsschritten, in denen der Agent vorgehen soll
#
# Vorteil:
# - Der nachfolgende Agent (Actor) weiß genau, welche Tools er in welcher Reihenfolge verwenden muss
# - Die Entscheidung, ob RAG oder calc benutzt wird, wird hier zentral getroffen

class Plan(BaseModel):
    intent: str = Field(..., description="Kurzbeschreibung der Aufgabe")
    needs_retrieval: bool = Field(..., description="Ob Kontext aus RAG benötigt wird")
    needs_calculation: bool = Field(..., description="Ob eine Berechnung nötig ist")
    queries: List[str] = Field(default_factory=list, description="Such-Queries für RAG")
    calc_expressions: List[str] = Field(default_factory=list, description="Mathe-Ausdrücke, falls nötig")
    subtasks: List[str] = Field(default_factory=list, description="Nummerierte Arbeitsschritte")


# 2. Prompt-Template `planner_prompt`
# -----------------------------------
# - Systemrolle: "Erzeuge einen knappen Plan als JSON"
# - Humanrolle: Übergibt die konkrete Nutzerfrage {question} und sagt, dass der Kontext aus der internen Bitcoin-Mining-PDF kommt
#
# Vorteil:
# - Wir trennen hier klar die Rollen (system vs. human), was die Modellsteuerung präziser macht
# - Das Modell wird gezwungen, nur JSON im `Plan`-Format zu liefern (durch späteren structured output)

planner_prompt = ChatPromptTemplate.from_messages([
    ("system", "Erzeuge einen knappen Plan als JSON (so kurz wie möglich)."),
    ("human", "Aufgabe: {question}\nKontext: interne PDF zu Bitcoin Mining.")
])


# 3. Verknüpfung Prompt → Modell (`planner_chain`)
# -----------------------------------------------
# - Wir verbinden das Prompt-Template mit unserem OpenAI-Modell (`model`)
# - Mit `.with_structured_output(Plan)` erzwingen wir, dass die Antwort direkt als Pydantic-Objekt geparst wird
# - Dadurch entfällt das manuelle JSON-Parsing
#
# Vorteil:
# - Automatisches Validieren der Felder (Pydantic)
# - Weniger Fehler bei der Weiterverarbeitung, weil die Struktur fix ist

planner_chain = planner_prompt | model.with_structured_output(Plan)
print("OK – Planner-Chain gebaut.")


OK – Planner-Chain gebaut.


# Actor/Agent (Tool-Calling, Schritt 2)

In [67]:
# ===== 3) ACTOR (Agent mit Tools) =====
# Der Actor ist die zweite Stufe unserer Agenten-Pipeline.
# Er übernimmt den vom Planner erstellten Plan und führt die nötigen Schritte mit den registrierten Tools aus.
#
# 1. Prompt-Template `agent_prompt`
# ---------------------------------
# - Systemrolle: Weist das Modell an, faktenbasiert zu antworten und Tools nur dann zu nutzen, wenn sie wirklich gebraucht werden.
#   Außerdem soll der "scratchpad"-Bereich (der interne Gedankenbereich des Agents) maximal 30 Wörter enthalten,
#   um Tokenverbrauch zu reduzieren.
# - Humanrolle: Platzhalter {input} – hier wird die eigentliche Anweisung oder Teilaufgabe des Agents eingefügt.
# - Placeholder `{agent_scratchpad}`: Dieser spezielle Platzhalter wird von LangChain intern genutzt,
#   um den aktuellen Stand der Zwischenergebnisse und Tool-Calls einzufügen.
#
# Vorteil:
# - Klare Trennung zwischen Nutzerinput und internen Agenten-Notizen
# - Minimierung des Tokenverbrauchs durch bewusst knappe Scratchpads

agent_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Antworte faktenbasiert. Nutze Tools nur falls nötig. "
     "Halte dein scratchpad extrem kurz (max. 30 Wörter)."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])


# 2. AgentExecutor `agent_exec`
# -----------------------------
# - `agent`: Der Agent selbst (wird vorher mit `create_openai_tools_agent(...)` gebaut)
# - `tools`: Liste der verfügbaren Tools (z. B. `rag_search`, `calc`)
# - `verbose=False`: Deaktiviert die ausführliche Konsolenausgabe für weniger Clutter
# - `max_iterations=2`: Beschränkt den Agenten auf maximal 2 Schritte, um unnötige Tool-Schleifen zu vermeiden
# - `early_stopping_method="generate"`: Falls das Limit erreicht ist, erzeugt der Agent trotzdem eine Antwort
#
# Vorteil:
# - Verhindert Endlosschleifen bei Tool-Aufrufen
# - Spart Tokens, weil unnötige Iterationen und zu lange Scratchpads vermieden werden

agent_exec = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    max_iterations=2,
    early_stopping_method="generate"
)


print("OK – AgentExecutor bereit.")


OK – AgentExecutor bereit.


# Formatter (Structured Output, Schritt 3)

In [68]:
# ===== 4) FORMATTER (Structured Output) =====
# Der Formatter ist die letzte Stufe in der Agenten-Pipeline.
# Er nimmt alle Zwischenergebnisse (Plan + Agenten-Trace) und wandelt sie in ein
# strukturiertes, einheitliches Antwortformat um, das leicht weiterverarbeitet werden kann.
#
# 1. Datenmodelle (Pydantic)
# --------------------------
# - `TableRow`: Einfaches Modell mit zwei Strings (key, value) für tabellarische Zusatzinfos.
# - `FinalAnswer`: Enthält alle Bestandteile der finalen Antwort:
#     - answer: Fließtext, kurz und präzise
#     - bullet_points: Liste der wichtigsten Erkenntnisse
#     - citations: Quellenangaben im Format [1], [2] → Verweis auf RAG-Snippets
#     - table: Liste von `TableRow`-Objekten, falls strukturierte Daten enthalten sind
#     - steps_taken: Liste kurzer Protokollpunkte zu den Schritten, die der Agent gemacht hat
#
# Vorteil:
# - Einheitliches Output-Format → keine Nachbearbeitung der rohen LLM-Antwort nötig
# - Struktur erleichtert maschinelle Weiterverarbeitung (Export, Anzeige, API-Response)


class TableRow(BaseModel):
    key: str
    value: str

class FinalAnswer(BaseModel):
    answer: str = Field(..., description="Kompakte, präzise Antwort in Fließtext.")
    bullet_points: List[str] = Field(default_factory=list, description="Wichtige Takeaways.")
    citations: List[str] = Field(default_factory=list, description="Referenzen wie [1], [2] die auf RAG-Snippets verweisen.")
    table: List[TableRow] = Field(default_factory=list, description="Optional: kleine Tabelle mit Kernwerten.")
    steps_taken: List[str] = Field(default_factory=list, description="Kurzprotokoll der Agentenschritte.")


# 2. Prompt-Template `formatter_prompt`
# -------------------------------------
# - Systemrolle: Weist das Modell an, die Antwort kompakt zu halten
#   (max. 80 Wörter in `answer`, max. 4 Bullet Points, max. 2 Quellen, max. 3 Tabellenzeilen).
#   Falls die Faktenlage unklar ist, soll das Modell explizit "Ich weiß es nicht." schreiben.
# - Humanrolle: Übermittelt die Eingabedaten für den Formatter:
#   - {question} → die Originalfrage
#   - {plan} → der vom Planner erstellte Kurzplan
#   - {trace} → gekürzte Agenten-Notizen inkl. Tool-Ausgaben
#
# Vorteil:
# - Garantierte Begrenzung der Antwortlänge (spart Tokens)
formatter_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Formatiere kompakt ins Schema. "
     "answer ≤ 80 Wörter, ≤ 4 bullets, ≤ 2 citations, ≤ 3 table rows. "
     "Wenn Evidenz fehlt: 'Ich weiß es nicht.'"),
    ("human",
     "Frage: {question}\nPlan(kurz): {plan}\nTrace(kurz):\n{trace}\nErzeuge das Ergebnis.")
])

# 3. Verkettung mit Modell (`formatter_chain`)
# --------------------------------------------
# - Das Prompt-Template wird mit `model.with_structured_output(FinalAnswer)` kombiniert.
# - Das sorgt dafür, dass das Modell direkt ein `FinalAnswer`-Objekt ausgibt, ohne dass man
#   selbst JSON-Parsing und Validierung implementieren muss.
#
# Vorteil:
# - Kein manuelles Parsen oder Validieren nötig
# - Output kommt garantiert im richtigen Format zurück
# WICHTIG: Prompt VOR Modell verkettet, dann structured output
formatter_chain = formatter_prompt | model.with_structured_output(FinalAnswer)
print("OK – Formatter-Chain gebaut.")


OK – Formatter-Chain gebaut.


# Orchestrator (Planner → Actor → Formatter)

In [69]:
# ===== ORCHESTRATOR: answer_with_agents =====
# Diese Funktion ist das zentrale Bindeglied zwischen allen Komponenten (Planner, Actor, Formatter).
# Sie koordiniert die komplette Pipeline, um aus einer Nutzerfrage eine strukturierte Antwort zu erzeugen.
#
# Ablauf:
# -------
# 1. Eingabe:
#    - `question`: Die Nutzerfrage als String.
#
# 2. Plan erstellen (Planner-Phase):
#    - Ruft `planner_chain.invoke(...)` auf, um aus der Frage einen strukturierten Plan
#      (Pydantic-Objekt vom Typ `Plan`) zu generieren.
#    - Dieser Plan enthält Flags, ob RAG-Suche oder Berechnung nötig ist, sowie die konkreten Queries/Expressions.
#
# 3. Zwischenspeicher (trace_parts):
#    - Initialisiert eine Liste `trace_parts`, in der alle Zwischenschritte protokolliert werden.
#    - Fügt zuerst den erzeugten Plan in gekürzter Form (via `_clip`) ein, um Tokenverbrauch zu reduzieren.
#
# 4. Kontext-Retrieval (RAG):
#    - Prüft, ob im Plan `needs_retrieval=True` steht und ob `queries` vorhanden sind.
#    - Falls ja: Führt nur die erste Query (`[:1]`) mit `agent_exec.invoke(...)` aus.
#      → Dadurch wird der Actor-Block aufgerufen, der das Tool `rag_search` benutzt.
#    - Fügt das (ggf. gekürzte) Ergebnis in den Trace ein.
#    - Falls keine RAG-Suche nötig ist, wird "(übersprungen)" protokolliert.
#
# 5. Berechnungen (CALC):
#    - Prüft, ob im Plan `needs_calculation=True` steht und ob `calc_expressions` vorhanden sind.
#    - Falls ja: Führt nur den ersten Ausdruck (`[:1]`) mit `agent_exec.invoke(...)` aus.
#      → Hier ruft der Actor das Tool `calc` auf.
#    - Fügt das (ggf. gekürzte) Ergebnis in den Trace ein.
#    - Falls keine Berechnung nötig ist, wird "(übersprungen)" protokolliert.
#
# 6. Trace zusammenfassen:
#    - Verbindet alle gesammelten Trace-Teile (`trace_parts`) zu einem String.
#    - Kürzt diesen String nochmals mit `_clip(..., 1400)`, um die Tokenmenge im Formatter-Aufruf gering zu halten.
#
# 7. Formatierung (Formatter-Phase):
#    - Ruft `formatter_chain.invoke(...)` auf und übergibt:
#        * Die Originalfrage
#        * Den Plan als Dictionary (`plan.model_dump()`)
#        * Den gekürzten Trace
#    - Ergebnis: Ein `FinalAnswer`-Objekt mit strukturierter, kompakter Antwort.
#
# Vorteil:
# - Kontrollierte Token-Nutzung durch gezieltes Kürzen (`_clip`)
# - Klare Trennung von Planung, Ausführung und Formatierung
# - Jeder Schritt ist einzeln austauschbar oder erweiterbar
#
# Sicherheitsmechanismen:
# - Maximal 1 Query und 1 Berechnung pro Anfrage (verhindert unkontrolliertes Wachstum)
# - Kürzungen bei Plan-, RAG- und CALC-Ausgaben
# - Gekappter Gesamtspeicher (`trace`) für den Formatter

def answer_with_agents(question: str) -> FinalAnswer:
    # Hilfsfunktion: Kürzt Strings auf n Zeichen und hängt "…" an, falls nötig
    def _clip(s, n): return (s[:n] + "…") if len(s) > n else s

    # 1) Plan erstellen
    plan: Plan = planner_chain.invoke({"question": question})

    # 2) Trace initialisieren
    trace_parts = []
    trace_parts.append(f"# Plan\n{_clip(plan.model_dump_json(indent=0), 500)}")

    # 3) Kontext-Retrieval (RAG)
    if plan.needs_retrieval and plan.queries:
        for q in plan.queries[:1]:  # nur die erste Query nutzen
            out = agent_exec.invoke({"input": f"Rufe rag_search auf für Query: {q}"})
            trace_parts.append(f"\n# RAG ({q})\n{_clip(out.get('output',''), 500)}")
    else:
        trace_parts.append("\n# RAG\n(übersprungen)")

    # 4) Berechnung (CALC)
    if plan.needs_calculation and plan.calc_expressions:
        for expr in plan.calc_expressions[:1]:  # nur den ersten Ausdruck nutzen
            out = agent_exec.invoke({"input": f"Berechne: {expr} (nutze calc)"})
            trace_parts.append(f"\n# CALC ({expr})\n{_clip(out.get('output',''), 150)}")
    else:
        trace_parts.append("\n# CALC\n(übersprungen)")

    # 5) Trace zusammenfassen und kürzen
    trace = _clip("\n".join(trace_parts), 1400)

    # 6) Formatierte Endausgabe erstellen
    final: FinalAnswer = formatter_chain.invoke({
        "question": question,
        "plan": plan.model_dump(),
        "trace": trace
    })
    return final


# Beispiele (laufen lassen & JSON ausgeben)

In [70]:
# Beispiel 1: Reine inhaltliche Frage → nutzt RAG
final1 = answer_with_agents(
    "Erkläre kurz die Vorteile von BTC-Mining in Verbindung mit Abwärmenutzung in Gewächshäusern."
)
print(final1.model_dump_json(indent=2))

{
  "answer": "BTC-Mining in Verbindung mit Abwärmenutzung in Gewächshäusern bietet mehrere Vorteile: Es schafft eine zusätzliche Einkommensquelle für Landwirte, indem Abwärme zur Beheizung der Gewächshäuser genutzt wird. Dies verbessert die Energieeffizienz und reduziert Betriebskosten. Zudem fördert es die Nachhaltigkeit, indem Abwärme, die sonst verloren ginge, sinnvoll eingesetzt wird. Die Synergie zwischen beiden Bereichen kann die Rentabilität steigern und die Abhängigkeit von fossilen Brennstoffen verringern.",
  "bullet_points": [
    "Zusätzliche Einkommensquelle für Landwirte",
    "Energieeffizienz durch Abwärmenutzung",
    "Reduzierung der Betriebskosten",
    "Nachhaltigkeit und Ressourcenschonung"
  ],
  "citations": [
    "[1]",
    "[2]"
  ],
  "table": [
    {
      "key": "Einkommensquelle",
      "value": "Zusätzliches Einkommen durch Mining"
    },
    {
      "key": "Energieeffizienz",
      "value": "Nutzung von Abwärme zur Beheizung"
    },
    {
      "key": "N

In [60]:
# Beispiel 2: Mit kleiner Rechnung (Tool calc) → Prompt-Chaining + Tools
final2 = answer_with_agents(
    "Wenn ein Miner 3 kW Abwärme liefert und ein Gewächshaus 9 kW Heizbedarf hat, "
    "wie viele Miner wären nötig? Erkläre kurz und nutze die PDF als Kontext."
)
print(final2.model_dump_json(indent=2))

{
  "answer": "Um den Heizbedarf eines Gewächshauses von 9 kW zu decken, sind 3 Miner erforderlich, da jeder Miner 3 kW Abwärme liefert. Die Berechnung lautet: 9 kW / 3 kW = 3 Miner.",
  "bullet_points": [
    "Heizbedarf Gewächshaus: 9 kW",
    "Abwärme pro Miner: 3 kW",
    "Benötigte Miner: 3",
    "Effiziente Nutzung von Abwärme."
  ],
  "citations": [
    "1",
    "2"
  ],
  "table": [
    {
      "key": "Heizbedarf Gewächshaus",
      "value": "9 kW"
    },
    {
      "key": "Abwärme pro Miner",
      "value": "3 kW"
    },
    {
      "key": "Benötigte Miner",
      "value": "3"
    }
  ],
  "steps_taken": [
    "Heizbedarf des Gewächshauses bestimmt.",
    "Abwärme eines einzelnen Miners ermittelt.",
    "Anzahl der benötigten Miner berechnet."
  ]
}
